In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/84.c7.0b.aa
(please check that this page displays the message 'small cows walk happily')


Check we're logged in

In [ ]:
user.wait_for_login()

True

### Upload a file for processing


In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

In [ ]:
creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

In [ ]:
hugs = Service(service_url="%s/hugs" % base_url)
par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
hugs

Service(url=hugs.acquire-aaai.com/t/hugs, uid=a0-a7)

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}

response = hugs.call_function(function="process", args=args)

In [ ]:
response

{'results': ['6f65ec37-e696-4c4e-8b33-6c58cf99c35d',
  '4bc9f7f6-535f-42ed-95b0-caf2590c1d71',
  '1c13b22b-6600-4097-85c0-5e5e828467ff',
  '91b861f8-06f6-423a-8ca4-a093dae5eb6a',
  '76706fa1-9ed2-4912-83fc-024de1bea11f',
  'f147b816-776f-4fe2-9821-c4ade880ff00',
  '09c19ef6-3fd2-4e4f-af16-accb54be9841',
  '8416c40e-208e-4de8-bd93-0cf799f0b4fb',
  'e481e5fc-b961-4b36-b04f-02e6f84b8493',
  '0bd57054-3bcc-46bf-a690-a45db3202ae9',
  'be817c02-9880-4c47-a425-129bfab80e85',
  '4f08b296-3bdf-4d2f-85be-5d6c6e14a9b9',
  '79228c52-cc58-4a01-9515-a2e54a474279',
  'dc4bcbcc-8099-4670-a763-408f95ac7ac8',
  'f5443428-552b-4b36-b729-5acc880f62e0']}

In [ ]:
from Acquire.ObjectStore import ObjectStore
from Acquire.Service import get_service_account_bucket

bucket = get_service_account_bucket()
for r in response["results"]:
    ObjectStore.delete_object(bucket=bucket, key=r)

ServiceError: You can only call this function from within a valid running service. A client cannot call this function.

In [ ]:
from HUGS.Processing import search

search_terms = widgets.Text(
                    value="Search",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="Locations",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

box = widgets.VBox(children=[search_terms, locations, data_type, search_button], layout=layout)

def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    x =  search(search_terms=search_terms.value, locations=locations.value, data_type=data_type.value)
    print(x)
    



In [ ]:
search_button.on_click(call_search)

In [ ]:
box